In [2]:
"""
@author: Jack Huang
"""
import os,time,math,signal,random,shutil,glob
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import input_data

In [3]:
class TestNN:
    def __init__(self, layers, w_rec, b_rec):
        self.layers = layers
        self.num_layers = len(self.layers)
        self.w_rec = w_rec
        self.b_rec = b_rec
        # TF placeholders and graph
        self.sess = tf.Session()
        self.input_tf = tf.placeholder(tf.float32, shape=[None, self.layers[0]])

        # Initialize NNs with trained weights and biases
        self.weights, self.biases = self.initialize_NN(self.layers,self.w_rec, self.b_rec)
        self.out_pred = tf.pow(10.,self.forward(self.input_tf))
        init = tf.global_variables_initializer()
        self.sess.run(init)   

    # Load the trained weights and baises to the NN
    def initialize_NN(self, layers, w_rec, b_rec):
        weights = []
        biases = []
        num_layers = len(layers)
        for i in range(0,num_layers-1):
            W = tf.Variable(w_rec[i])
            b = tf.Variable(b_rec[i])
            weights.append(W)
            biases.append(b)
        return weights,biases

    def log10(self, x):
        numerator = tf.log(x)
        denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
        return numerator / denominator
        
    # Forward NN 
    def forward(self, inputs):
        num_layers = len(self.weights) + 1 
        # h = inputs
        # h = (inputs - self.lb)/(self.ub - self.lb)
        h = self.log10(inputs)
        for l in range(num_layers - 2):
            W = self.weights[l]
            b = self.biases[l]
            h = tf.nn.sigmoid(tf.add(tf.matmul(h,W),b))
        W = self.weights[-1]
        b = self.biases[-1]
        return tf.add(tf.matmul(h,W),b)
    
    # Using NN to predict the output
    def predict(self, inputs):
        out_pred_value = self.sess.run(self.out_pred, {self.input_tf:inputs})
        return out_pred_value

In [4]:
# Get the weights and biases records 
def get_wb_rec(layers, rec_path):
    if os.path.exists(rec_path) == False:
        print('Oooops, No wb_rec found. Exiting ...')
        exit()
    num_layers = len(layers)
    w_rec = []
    b_rec = []
    for i in range(1, num_layers):
        temp_w_rec = np.load(rec_path+'w_rec'+str(i)+'.npy')
        temp_b_rec = np.load(rec_path+'b_rec'+str(i)+'.npy')
        w_rec.append(temp_w_rec)
        b_rec.append(temp_b_rec)
    return w_rec, b_rec 

In [5]:
if __name__ == "__main__":
    wraper = input_data.DataWrapper()
    # Layers must be exactly the same as fitNN.py

    layers_path = 'wb_rec/layers.npy'
    if os.path.exists(layers_path) != True:
        print('Ooops, no layers record found. Existing ...')
        exit()
    layers = np.load(layers_path)

    NN = [
      0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.09, \
      0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, \
      1, 2, 3, 4, 5, 6, 7, 8, 9, \
      10, 20, 30, 40, 50, 60, 70, 80, 90, \
      100, 200, 300, 400, 500, 
      600, 700, 800, 900, 1000]

    # To get the following data:
    # Sample in the log coordinates uniformly, 
    # then restore to the Cartesian coordinates
    ka = [
       1.12201845e-02, 1.25892541e-02, 1.41253754e-02, 1.58489319e-02,
       1.77827941e-02, 1.99526231e-02, 2.23872114e-02, 2.51188643e-02,
       2.81838293e-02, 3.16227766e-02, 3.54813389e-02, 3.98107171e-02,
       4.46683592e-02, 5.01187234e-02, 5.62341325e-02, 6.30957344e-02,
       7.07945784e-02, 7.94328235e-02, 8.91250938e-02, 1.00000000e-01,
       1.12201845e-01, 1.25892541e-01, 1.41253754e-01, 1.58489319e-01,
       1.77827941e-01, 1.99526231e-01, 2.23872114e-01, 2.51188643e-01,
       2.81838293e-01, 3.16227766e-01, 3.54813389e-01, 3.98107171e-01,
       4.46683592e-01, 5.01187234e-01, 5.62341325e-01, 6.30957344e-01,
       7.07945784e-01, 7.94328235e-01, 8.91250938e-01, 1.00000000e+00,
       1.12201845e+00, 1.25892541e+00, 1.41253754e+00, 1.58489319e+00,
       1.77827941e+00, 1.99526231e+00, 2.23872114e+00, 2.51188643e+00,
       2.81838293e+00, 3.16227766e+00, 3.54813389e+00, 3.98107171e+00,
       4.46683592e+00, 5.01187234e+00, 5.62341325e+00, 6.30957344e+00,
       7.07945784e+00, 7.94328235e+00, 8.91250938e+00, 1.00000000e+01,
       1.12201845e+01, 1.25892541e+01, 1.41253754e+01, 1.58489319e+01,
       1.77827941e+01, 1.99526231e+01, 2.23872114e+01, 2.51188643e+01,
       2.81838293e+01, 3.16227766e+01, 3.54813389e+01, 3.98107171e+01,
       4.46683592e+01, 5.01187234e+01, 5.62341325e+01, 6.30957344e+01,
       7.07945784e+01, 7.94328235e+01, 8.91250938e+01, 1.00000000e+02,
       1.12201845e+02, 1.25892541e+02, 1.41253754e+02, 1.58489319e+02,
       1.77827941e+02, 1.99526231e+02, 2.23872114e+02, 2.51188643e+02,
       2.81838293e+02, 3.16227766e+02, 3.54813389e+02, 3.98107171e+02,
       4.46683592e+02, 5.01187234e+02, 5.62341325e+02, 6.30957344e+02,
       7.07945784e+02, 7.94328235e+02, 8.91250938e+02, 1.00000000e+03]
    ka = np.array(ka)
    ka = ka[:,np.newaxis]

    NNs = [0]*len(NN)
    for i in range(len(NN)):
        NNs[i] = np.repeat([[NN[i]]], ka.shape[0], axis=0)
    # print(train_i,train_l)

    # Get the latest folder name 
    list_of_files = glob.glob('./wb_rec/*')
    latest_file = max(list_of_files, key=os.path.getctime)
    w_rec, b_rec = get_wb_rec(layers,latest_file+'/')
    model = TestNN(layers, w_rec, b_rec)

    pred_path = './predict/'

    for the_file in os.listdir(pred_path):
        file_path = os.path.join(pred_path, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
    if os.path.exists(pred_path) != True:
        os.makedirs(pred_path)
    for i in range(len(NNs)):
        inputs = np.concatenate((NNs[i],ka), axis=1)
        print(inputs,inputs.shape)
        time.sleep(60)
        out_value =  model.predict(inputs)
        print(out_value) 
        data_to_save = np.concatenate((ka,out_value),axis=1)
        np.savetxt(pred_path+str(NNs[i][0][0])+'_sk.csv', data_to_save)


[[1.00000000e-02 1.12201845e-02]
 [1.00000000e-02 1.25892541e-02]
 [1.00000000e-02 1.41253754e-02]
 [1.00000000e-02 1.58489319e-02]
 [1.00000000e-02 1.77827941e-02]
 [1.00000000e-02 1.99526231e-02]
 [1.00000000e-02 2.23872114e-02]
 [1.00000000e-02 2.51188643e-02]
 [1.00000000e-02 2.81838293e-02]
 [1.00000000e-02 3.16227766e-02]
 [1.00000000e-02 3.54813389e-02]
 [1.00000000e-02 3.98107171e-02]
 [1.00000000e-02 4.46683592e-02]
 [1.00000000e-02 5.01187234e-02]
 [1.00000000e-02 5.62341325e-02]
 [1.00000000e-02 6.30957344e-02]
 [1.00000000e-02 7.07945784e-02]
 [1.00000000e-02 7.94328235e-02]
 [1.00000000e-02 8.91250938e-02]
 [1.00000000e-02 1.00000000e-01]
 [1.00000000e-02 1.12201845e-01]
 [1.00000000e-02 1.25892541e-01]
 [1.00000000e-02 1.41253754e-01]
 [1.00000000e-02 1.58489319e-01]
 [1.00000000e-02 1.77827941e-01]
 [1.00000000e-02 1.99526231e-01]
 [1.00000000e-02 2.23872114e-01]
 [1.00000000e-02 2.51188643e-01]
 [1.00000000e-02 2.81838293e-01]
 [1.00000000e-02 3.16227766e-01]
 [1.000000

KeyboardInterrupt: 